# Pyomo

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/pyomo.ipynb)

Comme [OR Tools](https://github.com/google/or-tools), [Pyomo](https://github.com/Pyomo/pyomo) permet de formuler facilement des problèmes d'optimisation mathématique en Python et de transmettre ces problèmes à un solveur. Pyomo peut utiliser de nombreux solveurs : [glpk](https://www.gnu.org/software/glpk/) est gratuit et pratique pour les problèmes d'optimisation de petite et moyenne taille.

Docs:
- https://github.com/Pyomo/pyomo
- https://www.pyomo.org/
- https://www.gnu.org/software/glpk/

Voir aussi:
- https://jckantor.github.io/ND-Pyomo-Cookbook/README.html

In [32]:
import sys

# Supprimer les commentaires pour installer

# À installer dans tous les cas pour Google Colab
if 'google.colab' in sys.modules:
    !pip3 install -q -U pyomo

In [33]:
if sys.platform == 'linux' or sys.platform == 'linux2':
    !apt-get install -y -qq glpk-utils
elif sys.platform == 'darwin':
    # Make sure you have https://brew.sh/
    !brew install glpk
elif sys.platform == 'win32':
    print('Download and run the installer from https://winglpk.sourceforge.net/')

# Check install
!glpsol --help

==> Downloading https://formulae.brew.sh/api/formula.jws.json

==> Downloading https://formulae.brew.sh/api/cask.jws.json

To reinstall 5.0, run:
  brew reinstall glpk
Usage: glpsol [options...] filename

General options:
   --mps             read LP/MIP problem in fixed MPS format
   --freemps         read LP/MIP problem in free MPS format (default)
   --lp              read LP/MIP problem in CPLEX LP format
   --glp             read LP/MIP problem in GLPK format 
   --math            read LP/MIP model written in GNU MathProg modeling
                     language
   -m filename, --model filename
                     read model section and optional data section from
                     filename (same as --math)
   -d filename, --data filename
                     read data section from filename (for --math only);
                     if model file also has data section, it is ignored
   -y filename, --display filename
                     send display output to filename (for --math o

In [34]:
from pyomo.environ import ConcreteModel, Constraint, NonNegativeReals, maximize, Objective, SolverFactory, Var

## Premier problème d'optimisation linéaire

Un fabricant d'électroménager tente de maximiser ses profits en décidant du nombre de lave-linges et de sèche-linge à produire, sous réserve de contraintes sur les ressources de fabrication, de test et d'assemblage:
- Les ressources de l’entreprise sont telles que seulement 20 heures de fabrication, 20 heures d’assemblage et 25 heures de tests sont disponibles par jour.
- La production d'un lave-linge nécessite 1 heure de fabrication, 2 heures d'assemblage et 2 heures de tests pour générer un bénéfice de €100,00.
- La production d'un sèche-linge nécessite 2 heures de fabrication, 1 heure d'assemblage et 2 heures de tests pour générer un bénéfice de €120,00.

Nous avons formulé mathématiquement ce problème comme suit, en définissant
- $x_1$ comme le nombre de lave-linges produites par jour, et
- $x_2$ comme le nombre de sèche-linges produits par jour.

$$
\begin{aligned}
& \underset{x}{\text{maximiser}}
& & 100 x_1 + 120 x_2\\
& \text{sachant que}
& & x_1 + 2 x_2 \leq 20 \\
& & & 2x_1 +  x_2 \leq 20 \\
& & & 2x_1 + 2 x_2 \leq 25 \\
&&& x_1, x_2 \geq 0
\end{aligned}
$$

In [35]:
# Créer le modèle
model = ConcreteModel()

# Déclarer les 2 variables x1 et x2
model.x = Var([1,2],domain=NonNegativeReals)

# Déclarer l'objectif
model.profit = Objective(expr = 100*model.x[1] + 120*model.x[2], sense=maximize)

# Déclarer les contraintes
model.manufacturing = Constraint(expr = model.x[1] +2*model.x[2] <= 20)
model.assembly = Constraint(expr = 2*model.x[1] + model.x[2] <= 20)
model.testing = Constraint(expr = 2*model.x[1] + 2*model.x[2] <= 25)

# Afficher le modèle
model.pprint()

1 Var Declarations
    x : Size=2, Index={1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 100*x[1] + 120*x[2]

3 Constraint Declarations
    assembly : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  -Inf : 2*x[1] + x[2] :  20.0 :   True
    manufacturing : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  -Inf : x[1] + 2*x[2] :  20.0 :   True
    testing : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper : Active
        None :  -Inf : 2*x[1] + 2*x[2] :  25.0 :   True

5 Declarations: x profit manufacturing assembly testing


In [36]:
# Résoudre le modèle avec le solveur glpk
executable = 'glpsol' #'/usr/bin/glpsol'
SolverFactory('glpk', executable=executable).solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1400.0
  Upper bound: 1400.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of nonzeros: 6
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.009205818176269531
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

In [37]:
# Affichage de la solution
def print_solution(model):
    print('Profit quotidien = ', model.profit())
    print('Nombre de lave-linges  x1 = ', model.x[1].value)
    print('Nombre de sèche-linges x2 = ', model.x[2].value)
print_solution(model)

Profit quotidien =  1400.0
Nombre de lave-linges  x1 =  5.0
Nombre de sèche-linges x2 =  7.5


In [38]:
# Affichage des ressources utilisées
def print_resources(model):
    print('Ressources utilisées')
    print('Fabrication  = ', model.manufacturing())
    print('Assemblage = ', model.assembly())
    print('Test = ', model.testing())
print_resources(model)

Ressources utilisées
Fabrication  =  20.0
Assemblage =  17.5
Test =  25.0


On note que la production est limitée par la fabrication et le test, puisque les 20 heures d'assemblage disponibles quotidiennement ne sont pas utilisées.

In [39]:
# Déactiver les contraintes limitatives
model.manufacturing.deactivate()
model.testing.deactivate()

# Augmenter les ressources disponibles
model.manufacturing_new = Constraint(expr = model.x[1] +2*model.x[2] <= 21 )
model.testing_new = Constraint(expr = 2*model.x[1] +2*model.x[2] <= 26 )

# Résoudre le modèle avec le solveur glpk
SolverFactory('glpk', executable=executable).solve(model)

# Afficher la solution
print_solution(model)
print('\n')
# Afficher les ressources utilisées
print_resources(model)

Profit quotidien =  1460.0
Nombre de lave-linges  x1 =  5.0
Nombre de sèche-linges x2 =  8.0


Ressources utilisées
Fabrication  =  21.0
Assemblage =  18.0
Test =  26.0
